In [33]:
import torch
import torchvision
import cv2
from PIL import Image
from torchvision import transforms as T
import numpy as np

In [34]:
if torch.cuda.is_available():
    device = torch.device('cuda')
# if torch.backends.mps.is_available():
#     device = torch.device("mps")
else:
    device = torch.device('cpu')

In [35]:
# # Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to(device)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-22 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [36]:
!pip install deep-sort-realtime

In [37]:
from deep_sort_realtime.deepsort_tracker import DeepSort

In [38]:
object_tracker = DeepSort(max_iou_distance=0.2,max_age=250,nms_max_overlap = 0.05,gating_only_position=True,n_init=2,max_cosine_distance=0.9)

In [39]:
confidence_threshold = 0.5
object_type = ['car']
nms_threshold = 0.3

In [40]:
import time
count = 0
lst = []

In [41]:
count = 0
lst = []
car_id = 2
confidence_threshold = 0.6
object_type = ['car']

cap = cv2.VideoCapture('try.mp4')

width = int(cap.get(3))
height = int(cap.get(4))


out = cv2.VideoWriter('try.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))

model.eval()

while cap.isOpened():
    # Read the next frame
    ret, frame = cap.read()
    start = time.perf_counter()
    # Check EOF
    if not ret:
        print("NOPE")
        break
    #prediction YOLO
    with torch.no_grad():
      pred = model(frame)

    detections = []
    for detection in pred.xyxy[0]:
        class_id = int(detection[5])
        confidence = detection[4]
        if confidence> confidence_threshold and class_id ==car_id:
            x_min,y_min,x_max,y_max = detection[:4].cpu().detach().numpy().astype('int')
            x,y,w,h = [x_min,y_min,int(x_max-x_min),int(y_max-y_min)]
            cv2.rectangle(frame,(int(x_min), int(y_min)),(int(x_max), int(y_max)),(0,0,255),2)
            detections.append(([x,y,w,h],confidence,'car'))

    tracks = object_tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        bbox = ltrb


    count = object_tracker.tracker._next_id-1
    cv2.putText(frame, f'Count: {int(count)}', (20,200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

    out.write(frame)
out.release()
print(count)

NOPE
0
